In [1]:
from glom import glom
from elasticsearch import Elasticsearch, helpers
import eland as ed
import pandas as pd
from itertools import chain
from collections import Counter

In [2]:
# es = Elasticsearch(
#     "https://elasticsearch:9200",
#     api_key="YjVwMHpKSUI1aVdrRE5nRHhQN0o6UkxLTGRoUGRSVDZic3NCU2IzNFVnQQ==",
#     verify_certs=False,
# )
Elasticsearch("http://elasticsearch:9200", verify_certs=False, basic_auth=("elastic", "123456"))

/usr/local/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:400: SecurityWarning: Connecting to 'https://elasticsearch:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [4]:
es.info()

/usr/local/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': '75a29ff81cd8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'nLCYgzN-QquM4_GXqy3jyQ', 'version': {'number': '8.15.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '253e8544a65ad44581194068936f2a5d57c2c051', 'build_date': '2024-09-02T22:04:47.310170297Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
es.indices.get()

TypeError: IndicesClient.get() missing 1 required keyword-only argument: 'index'

In [73]:
q_word = "A酸"
body = {
    "_source": False,
    "query": {
        "nested": {
            "path": "context_aste",
            "query": {
                "match": {
                    "context_aste.a": q_word,
                }
            },
            "inner_hits": {},
        },
        "nested": {
            "path": "title_aste",
            "query": {
                "match": {
                    "title_aste.a": q_word,
                }
            },
            "inner_hits": {},
        },
        "nested": {
            "path": "comments.content_aste",
            "query": {
                "match": {
                    "comments.content_aste.a": q_word,
                }
            },
            "inner_hits": {},
        },
    },
}


In [104]:
tmp = es.search(
    index="dcard",
    body=body,
    filter_path=["hits.hits.inner_hits.*.hits.hits._source.*", "hits.total.value"],
)
tmp = list(chain.from_iterable(glom(tmp.body, "hits.hits.**.hits.hits.*._source")))

/usr/local/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'elasticsearch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [142]:
tag_counter = Counter(tmp["t"])

pos_o_counter = Counter(tmp.o[tmp.t == "POS"])
neg_o_counter = Counter(tmp.o[tmp.t == "NEG"])
nat_o_counter = Counter(tmp.o[tmp.t == "NAT"])

In [143]:
tag_counter.items()

dict_items([('NAT', 4), ('POS', 1)])

In [144]:
def aop_df_2_data(aop_df):
    tag_counter = Counter(aop_df["t"])
    return [
        {
            "value": v,
            "name": k,
            "children": [
                {"value": c, "name": w}
                for w, c in Counter(aop_df.o[aop_df.t == k]).items()
            ],
        }
        for k, v in tag_counter.items()
    ]

In [145]:
aop_df_2_data(tmp)

[{'value': 4,
  'name': 'NAT',
  'children': [{'value': 1, 'name': '口服'},
   {'value': 1, 'name': '減少'},
   {'value': 1, 'name': '去'},
   {'value': 1, 'name': '吃'}]},
 {'value': 1, 'name': 'POS', 'children': [{'value': 1, 'name': '好用'}]}]